In [1]:
# Import and set all required dependencies
import json
import matplotlib.pyplot as plt
import pandas as pd
import requests
import gmaps
from api_keys import g_key

In [2]:
city_df = pd.read_csv("weather_data.csv")
city_df.dropna()
city_df.head()

ID             City  Cloudiness Country        Date  Humidity  Latitude  \
0   0      Port Alfred         100      ZA  1618223843        75  -33.5906   
1   1      Provideniya         100      RU  1618223843        89   64.3833   
2   2  Tiznit Province          78      MA  1618223844        39   29.5833   
3   3            Bluff          70      NZ  1618223845        93  -46.6000   
4   4        Coyhaique          98      CL  1618223845        93  -45.5752   

   Longitude  Max Temperature  Wind Speed  
0    26.8910            74.35       14.12  
1  -173.3000             9.73       22.88  
2    -9.5000            72.88        7.14  
3   168.3333            54.00        1.01  
4   -72.0662            51.80        2.30

In [3]:
gmaps.configure(api_key=g_key)

# Store 'Lat' and 'Lng' from DF into lists
locations = city_df[["Latitude", "Longitude"]]
humidity = city_df["Humidity"].astype(float)

In [5]:
# Heatmap
fig = gmaps.figure(center = [0,0] ,zoom_level = 2)

# Add heat layer over map 
heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                               dissipating=False, max_intensity=100,
                               point_radius = 4)
fig.add_layer(heat_layer)
#Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
# configuration of perfect conditions (my perfect conditions)
temp_file=city_df.loc[(city_df['Max Temperature'] > 70)&(city_df['Max Temperature'] < 80)]
wind_file=temp_file.loc[(temp_file['Wind Speed'] < 10)]
ideal_weather=wind_file.loc[(wind_file['Cloudiness'] == 0)]
ideal_weather.head()

ID                   City  Cloudiness Country        Date  Humidity  \
5      5        Arraial do Cabo           0      BR  1618223846        83   
56    56              Cape Town           0      ZA  1618223879        47   
66    66                  Sīwah           0      EG  1618223890        21   
105  105                Pajapan           0      MX  1618223915        81   
149  149  Los Llanos de Aridane           0      ES  1618223942        73   

     Latitude  Longitude  Max Temperature  Wind Speed  
5    -22.9661   -42.0278            75.20        5.75  
56   -33.9258    18.4232            77.00        5.75  
66    29.2041    25.5195            71.62        5.53  
105   18.2500   -94.7000            72.01        7.47  
149   28.6585   -17.9182            71.60        3.44

In [14]:
# create hotel file
hotel_df = ideal_weather.reset_index(drop=True)
hotel_df["Hotel Name"] = ""

# parameters
params = {
    "radius": 5000,
    "types": "hotel",
    "keyword": "hotels",
    "key": g_key
}

# Use the lat/lng we recovered to identify hotels
for index, row in hotel_df.iterrows():
    
    # latitudes and longitudes from df
    lat = row["Latitude"]
    lng = row["Longitude"]
    
    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotels" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # make request and print url
    hotel_name = requests.get(base_url, params=params)
    
    # convert to json
    hotel_name = hotel_name.json()
    print(json.dumps(hotel_name, indent=4, sort_keys=True))
    
    # Since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_name["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Billing on the Google Cloud Project at https://console.cloud.google.com/project/_/billing/enable Learn more at https://developers.google.com/maps/gmp-get-started",
    "html_attributions": [],
    "results": [],
    "status": "REQUEST_DENIED"
}
Missing field/result... skipping.
{
    "error_message": "You must enable Bil

ID                   City  Cloudiness Country        Date  Humidity  \
0     5        Arraial do Cabo           0      BR  1618223846        83   
1    56              Cape Town           0      ZA  1618223879        47   
2    66                  Sīwah           0      EG  1618223890        21   
3   105                Pajapan           0      MX  1618223915        81   
4   149  Los Llanos de Aridane           0      ES  1618223942        73   
5   176               Celestún           0      MX  1618223959        84   
6   196               Pochutla           0      MX  1618223973        80   
7   214       Puerto Escondido           0      MX  1618223984        77   
8   320        Lázaro Cárdenas           0      MX  1618224080        86   
9   375                 Weinan           0      CN  1618224121        37   
10  477             Vila Velha           0      BR  1618224181        88   

    Latitude  Longitude  Max Temperature  Wind Speed Hotel Name  
0   -22.9661   -42.0278            75.20        5.75             
1   -33.9258    18.4232            77.00        5.75             
2    29.2041    25.5195            71.62        5.53             
3    18.2500   -94.7000            72.01        7.47             
4    28.6585   -17.9182            71.60        3.44             
5    20.8667   -90.4000            77.20        8.50             
6    15.7432   -96.4661            75.99        1.99             
7    15.8500   -97.0667            75.90        7.16             
8    17.9583  -102.2000            70.20        4.36             
9    34.5036   109.5089            72.00        6.71             
10  -20.3297   -40.2925            75.00        3.44

In [9]:
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [11]:
# Marker added to mark hotels
markers = gmaps.marker_layer(locations,
    info_box_content=[f"Hotel loc: {rate}" for rate in hotel_info])

fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))